<div>
<center><img src="Flux-logo.svg" width="400"/>
</div>

# Chapter 2: Using Flux to manage and deploy distributed services

Now that we have learned about hierarchical scheduling and its benefits, let's dive deeper into the structure of the individual Flux instances that comprise a hierarchy and examine how that structure enables the management and deployment of distributed services. In this module, we cover:
1. The structure of Flux instances
2. Management of Flux services
3. Examples of services in Flux (`flux kvs` and `flux archive`)

## The structure of Flux instances

As mentioned in [Chapter 2](./01_flux_tutorial.ipynb), a Flux instance is comprised of one or more Flux brokers. A high-level depiction of the design of a Flux broker is shown in the figure below.

<figure>
<img src="img/flux-broker-design.png">
<figcaption>
<i>Image created by Ian Lumsden for the Flux tutorials</i></figcaption>
</figure>

Each broker is a program built on top of the ∅MQ networking library. The broker contains two main components. First, the broker implements Flux-specific networking abstractions over ∅MQ, such as remote-proceedure call (RPC) and publication-subscription (pub-sub). Second, the broker contains several core services, such as PMI (for MPI support), run control support (for enabling automatic startup of other services), and, most importantly, broker module management. The remainder of a Flux broker's functionality comes from broker modules: specially designed services that the broker can deploy in independent OS threads. Some examples of broker modules provided by Flux include:
* Job scheduling (both [traditional and hierarchical](./02_flux_scheduling.ipynb))
* Fluxion (Flux's advanced graph-based scheduler)
* Banks and accounting (for system-wide deployments of Flux)
* PMIx (for OpenMPI)
* An in-memory content store (useful for preloading data into pods on cloud)

When Flux starts, it launches one or more brokers across the resources it manages. By default, Flux will launch one broker per node, but this can be configured (e.g., with the `--test-size` flag to `flux start` shown in [Chapter 1](./01_flux_tutorial.ipynb)). After launching the brokers, Flux will designate one broker as the "leader" and the rest as "followers". The leader serves as entrypoint into the Flux instance, and it serves as the starting point for most Flux commands. The distribution of brokers and the "leader-follower" designations are shown in the following figure:

<figure>
<img src="img/flux-instance-pre-tbon.png">
<figcaption>
<i>Image created by Vanessa Sochat for Flux Framework Components documentation</i></figcaption>
</figure>

After launching the brokers and designating a leader, Flux uses the brokers' network abstractions to connect the brokers together into what we call the "tree-based overlay network", or TBON for short. This network is shown in the figure below. This overlay network connects brokers together in a pre-defined tree-based topology (e.g., *k*-ary and binomial). Whenever brokers or instances of distributed services running on top of the brokers need to communicate, they can send messages up and down this tree-structured network. This tree-structured network is used over alternative designs (e.g., all-to-all networks used by MPI) because it provides better scalability (by minimizing communication), security, and fault tolerance for a service-focused framework. More information about these benefits and Flux's overall design can be found in our [publications](https://flux-framework.org/publications/) (particularly our [2014 paper on Flux](https://ieeexplore.ieee.org/document/7103433) presented at ICPP).

<figure>
<img src="img/flux-instance-w-tbon.png">
<figcaption>
<i>Image created by Vanessa Sochat for Flux Framework Components documentation</i></figcaption>
</figure>

### How Flux instances support services

Services in Flux are implemented as broker modules that can be deployed across one or more brokers. Once deployed, these services can leverage the other components of the broker, including message routing over the TBON and services provided by other broker modules. As a result, broker modules allow for the creation of composable, easily deployable services for Flux instances.

## Management of Flux services

To manage and query services, Flux provides the `flux module` command. The sub-commands provided by `flux module` can be seen by running the cell below.

In [ ]:
!flux module --help

While going through [Module 2](./02_flux_scheduling.ipynb), we've already encountered some built-in services provided by Flux, such as:
* `job-ingest` (used by Flux submission commands like `flux batch` and `flux run`)
* `job-list` (used by `flux jobs`)
* `sched-fluxion-qmanager` (used by `flux tree`)
* `sched-fluxion-resource` (also used by `flux tree`)

We can see that these services are loaded and available by running the cell below.

In [ ]:
!flux module list

Users and system administrators can easily load and unload services using the `flux module load` and `flux module remove` commands. To show this, let's unload Fluxion (Flux's graph-based scheduler) and replace it with the built-in simple scheduler.

In [ ]:
!flux module remove sched-fluxion-qmanager
!flux module remove sched-fluxion-resource
!flux module load sched-simple
!flux module list

In this code block, we unload the 2 services that comprise Fluxion: `sched-fluxion-qmanager` and `sched-fluxion-resource`. Next, we load the simple scheduler (`sched-simple`), and, finally, we look at the running servicees. We now see that Fluxion is not available, and the simple scheduler is.

Next, let's reload Fluxion, but, this time, let's pass some extra arguments to specialize our Flux instance. In particular, we will limit the scheduling depth to 4 and populate Fluxion's resource graph with:
* Nodes
* Sockets
* Cores

In [ ]:
# Run flux dmesg to make sure sched-simple has no more work before unloading
!flux dmesg -C
!flux module remove sched-simple
!flux module load sched-fluxion-resource load-allowlist=node,socket,core
!flux module load sched-fluxion-qmanager queue-params=queue-depth=4
!flux module list

## Examples of services in Flux

In this section, we will cover two services that expand Flux's usefulness to diverse applications:
1. `flux kvs`
2. `flux archive`

### flux kvs

One of the core services built into Flux is the key-value store (KVS). It is used in many other services, including most of Flux's resource management services, the `flux archive` service below, and DYAD (which we will explore in [Supplementary Chapter 1](./supplementary/dyad/dyad_dlio.ipynb)). These services use the KVS to persistantly store information and retrieve it later (potentially after a restart of Flux).

The `flux kvs` command provides a utility to list and manipulate values of the KVS. As a example of using `flux kvs`, let's use the command to examine information saved by the `resource` service.

In [ ]:
!flux kvs ls
!flux kvs ls resource
!flux kvs get resource.R | jq

The KVS is such an essential component of Flux that we provide C and Python APIs to interact with it. To learn more about interacting with the KVS from these languages, take a look at these documentation pages:
* C's `flux_kvs_commit` [family of functions](https://flux-framework.readthedocs.io/projects/flux-core/en/latest/man3/flux_kvs_commit.html)
* C's `flux_kvs_copy` [family of functions](https://flux-framework.readthedocs.io/projects/flux-core/en/latest/man3/flux_kvs_copy.html)
* C's `flux_kvs_getroot` [family of functions](https://flux-framework.readthedocs.io/projects/flux-core/en/latest/man3/flux_kvs_getroot.html)
* C's `flux_kvs_lookup` [family of functions](https://flux-framework.readthedocs.io/projects/flux-core/en/latest/man3/flux_kvs_lookup.html)
* C's `flux_kvs_namespace_create` [family of functions](https://flux-framework.readthedocs.io/projects/flux-core/en/latest/man3/flux_kvs_namespace_create.html)
* C's `flux_kvs_txn_create` [family of functions](https://flux-framework.readthedocs.io/projects/flux-core/en/latest/man3/flux_kvs_txn_create.html)
* Python's `flux.kvs` [module](https://flux-framework.readthedocs.io/projects/flux-core/en/latest/python/autogenerated/flux.kvs.html#module-flux.kvs)

### flux archive 📚️

As Flux is used more in cloud environments, we might find ourselves in a situation where we have a cluster without a shared filesystem. The `flux archive` command helps with this situation. At a high level, `flux archive` allows us to save named pieces of data (e.g., files) to the Flux KVS for later retrieval.

When using `flux archive`, we first have to create an named archive. In the code below, we will create a text file and then save it into an archive using `flux archive`. Note that, for larger files, you can speed up the creation and extraction of archives by using the `--mmap` flag.

In [ ]:
!echo "Sweet dreams 🌚️ are made of cheese, who am I to diss a brie? 🧀️" > shared-file.txt
!flux archive create --name myarchive --directory $(pwd) shared-file.txt

When we run this code, we are creating an archive in the leader broker. Now that the archive is created, we will want to extract its contents onto the other nodes of our cluster. To do this, we first need to ensure that the directory that we will extract into exists on those nodes. This can be done using `flux exec`. The `flux exec` command will execute a command on the nodes associated with specified brokers. Let's use `flux exec` to run `mkdir` on all the nodes of our cluster except the leader broker's node.

In [ ]:
!flux exec -r all -x 0 mkdir -p $(pwd)

The flags provided to `flux exec` do the following:
* `-r all`: run across all brokers in the Flux instance
* `-x 0`: don't runn on broker 0 (i.e., the leader broker)

Now that the directory has been created on all our nodes, we can extract the archive onto those nodes by combining `flux exec` and `flux archive extract`.

In [ ]:
!flux exec -r all -x 0 flux archive extract --name myarchive --directory $(pwd) shared-file.txt

Finally, when we're done with the archive, we can remove it with `flux archive remove`.

In [ ]:
!flux archive remove --name myarchive

Finally, note that `flux archive` was named `flux filemap` in earlier versions of Flux.

`flux kvs` and `flux archive` are two useful, but simple exammples of Flux services. Flux also supports more complex services, including services for runtime data movement, such as DYAD (covered in [Supplementary Chapter 1](./supplementary/dyad/dyad_dlio.ipynb)).

# This concludes Chapter 2.

In this module, we covered:
1. The structure of Flux instances and how that structure enables distributed services (including traditional and hierarchical scheduling)
2. How to start and stop services in Flux
3. Two useful services for users of Flux (i.e., `flux kvs` and `flux archive`)

To finish the tutorial, open [Chapter 3](./03_flux_tutorial_conclusions.ipynb).